confusion matrix :
    http://gcucurull.github.io/caffe/python/deep-learning/2016/06/29/caffe-confusion-matrix/
    https://github.com/gcucurull/caffe-conf-matrix
FCN :
    score :
        https://github.com/shelhamer/fcn.berkeleyvision.org/blob/master/score.py
    infer :
        https://github.com/shelhamer/fcn.berkeleyvision.org/blob/master/infer.py

In [1]:
from __future__ import division
from __future__ import print_function
import caffe
import numpy as np
import os
import sys
from datetime import datetime
from PIL import Image
from IPython.display import Image
from IPython.display import clear_output
import time

In [2]:
def log_progress(sequence, title='', every=1, size=None, end_msg='', removeAtTheEnd=False):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{title} : {value} ({index} / {size})'.format(
                        title=title,
                        value=sequence[index-1],
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = end_msg
        if removeAtTheEnd:
            box.close()
    

In [3]:
def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n, n)

In [4]:
import _init_paths
import caffe
#import jg_layers.jg_input_voc_layers

In [5]:
def evaluate(results):
    
    hist_cumul = 0*results[0]
    for hist in results:
        hist_cumul+=hist
    result = {}
    

    acc = np.diag(hist_cumul).sum() / hist_cumul.sum()
    # per-class accuracy
    pc_acc = np.diag(hist_cumul) / hist_cumul.sum(1)
    mean_accuracy= np.nanmean(pc_acc)
    # per-class IU
    iu = np.diag(hist_cumul) / (hist_cumul.sum(1) + hist_cumul.sum(0) - np.diag(hist_cumul))
    mean_iu= np.nanmean(iu)
    freq = hist_cumul.sum(1) / hist_cumul.sum()
    freq_weighted_iu=(freq[freq > 0] * iu[freq > 0]).sum()
    
    print("Accuracy :",acc)
    print("per-class accuracy :",mean_accuracy)
    print("per-class IU :",mean_iu)
    print("per-class IU (frequency weighted) :",freq_weighted_iu)
    
    return acc

In [6]:
#newParams = {}

## classes
#newParams['classes_file_path']= '/home/jogue/workspace/datasets/SUNRGBD_pv/37_classes.txt'

## images
#newParams['images_folder']= '/home/jogue/workspace/datasets/SUNRGBD_pv/data/rgb_i_100_8bits/'
#newParams['image_file_extension']= 'png'
## segmentations
#newParams['segmentations_folder']= '/home/jogue/workspace/datasets/SUNRGBD_pv/data_nyudv2/labels_segmentation_37d_range_10-10-100/testGupta/'                            
#newParams['segmentation_file_extension']= 'png'

##sets
#newParams['set_file_path']= '/home/jogue/workspace/datasets/SUNRGBD_pv/data/sets/nyudv2_gupta/testGupta_10-10-100.txt'

## shuffle
#newParams['shuffle']= False

In [7]:
ICIPmodels = '/home/jogue/workspace/ICIP/models/'
prototxt_path = ICIPmodels+'jg_eval.prototxt'
caffe_gpu = True
if not caffe_gpu:
    caffe.set_mode_cpu()
else:
    caffe.set_mode_gpu()
    caffe.set_device(0);

In [8]:
net = caffe.Net(prototxt_path, caffe.TEST)
#net = caffe.Net(prototxt_path,weights_path , caffe.TEST)

Number of image : 654
Separated in 37 classes : {'toilet': 32, 'whiteboard': 29, 'wall': 0, 'night_stand': 31, 'books': 22, 'mirror': 18, 'table': 6, 'chair': 4, 'floor': 1, 'tv': 24, 'lamp': 34, 'window': 8, 'curtain': 15, 'blinds': 12, 'dresser': 16, 'picture': 10, 'ceiling': 21, 'door': 7, 'shower_curtain': 27, 'fridge': 23, 'cabinet': 2, 'sink': 33, 'desk': 13, 'bookshelf': 9, 'towel': 26, 'box': 28, 'floor_mat': 19, 'shelves': 14, 'sofa': 5, 'counter': 11, 'bed': 3, 'person': 30, 'paper': 25, 'bag': 36, 'bathtub': 35, 'pillow': 17, 'clothes': 20}


#### Blobs & Layers

In [9]:
B = [(k, v.data.shape) for k, v in net.blobs.items()]
L = [(k, v[0].data.shape, v[1].data.shape) for k, v in net.params.items() if k != 'upscore37']

#### Updating with new params

In [10]:
modelToTest = ['10-10-100','60-10-100','80-10-100'] #,100,90,80,70,60,40,20]
#modelToTest = ['10-10-100','60-10-100','80-10-100',100,90,80,70,60,40,20]
#DatasetToTest = range(100,0,-10)
for luminosity_model in log_progress(modelToTest,title='Luminosity model', end_msg='Finished !'): 
    weights_path = ICIPmodels+'rgb_'+str(luminosity_model)+'/weights.caffemodel'
    net.copy_from(weights_path)
    for luminosity_datatest in log_progress(range(100,0,-20), title='Luminosity dataset', removeAtTheEnd=True):
        newParams = {}
        newParams['images_folder']= '/home/jogue/workspace/datasets/SUNRGBD_pv/data/rgb_i_'+str(luminosity_datatest)+'_8bits/'
        newParams['iter_counter']=0
        newParams['countEpoch']=0
        net.layers[0].update(newParams)
        
        results = []
        losses = []
        
        n_cl = net.blobs['score'].channels
        hist = np.zeros((n_cl, n_cl))
        loss = 0
        print("###############################################################")
        print("NEW TESTING : model",luminosity_model,"% on",luminosity_datatest,"%.")
        #for idx in log_progress(range(net.layers[0].nb_images), title='iteration', end_msg=str(i),removeAtTheEnd=True):
        for idx in log_progress(range(net.layers[0].nb_images), title='iteration', removeAtTheEnd=True):
            net.forward()
            score_map = net.blobs['score'].data[0].argmax(0)
            gt_map = net.blobs['label'].data[0, 0]
            hist = fast_hist(gt_map.flatten(),score_map.flatten(),n_cl)
            # compute the loss as well
            loss = net.blobs['loss'].data.flat[0]
            results.append(hist)
            losses.append(loss)
        print("Images processed =",net.layers[0].iter_counter)
        print("Cumulative loss =",sum(losses))
        evaluate(results)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


###############################################################
NEW TESTING : model 10-10-100 % on 100 %.
New epoch [total epoch : 1 ].
Images processed = 0
Cumulative loss = 906.887292072
Accuracy : 0.672508756813
per-class accuracy : 0.504340923923
per-class IU : 0.354876512169
per-class IU (frequency weighted) : 0.522990298425
###############################################################
NEW TESTING : model 10-10-100 % on 80 %.


/home/jogue/soft/anaconda2/envs/jg_pfr/lib/python2.7/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in true_divide
/home/jogue/soft/anaconda2/envs/jg_pfr/lib/python2.7/site-packages/ipykernel/__main__.py:14: RuntimeWarning: invalid value encountered in true_divide
Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


New epoch [total epoch : 1 ].
Images processed = 0
Cumulative loss = 890.431005761
Accuracy : 0.657902757314
per-class accuracy : 0.487240939238
per-class IU : 0.345153866385
per-class IU (frequency weighted) : 0.506962673916
###############################################################
NEW TESTING : model 10-10-100 % on 60 %.


Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


...

New epoch [total epoch : 1 ].
Images processed = 0
Cumulative loss = 1043.16543667
Accuracy : 0.597304353484
per-class accuracy : 0.385016654531
per-class IU : 0.281594934037
per-class IU (frequency weighted) : 0.434231963796
###############################################################
NEW TESTING : model 80-10-100 % on 40 %.


Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


New epoch [total epoch : 1 ].
Images processed = 0
Cumulative loss = 1241.45524731
Accuracy : 0.507927838799
per-class accuracy : 0.255891724902
per-class IU : 0.185649235221
per-class IU (frequency weighted) : 0.33274805095
###############################################################
NEW TESTING : model 80-10-100 % on 20 %.


Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


New epoch [total epoch : 1 ].
Images processed = 0
Cumulative loss = 1451.07231179
Accuracy : 0.433026004821
per-class accuracy : 0.146026516343
per-class IU : 0.0976630980738
per-class IU (frequency weighted) : 0.247910150091
